In [ ]:
%%capture
%pylab inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

import os,time,copy
import numpy as np
import pandas as pd
import copy
from pathlib import Path

In [ ]:
# local settings
basepath = Path().absolute().parent

# Add repository's src folder to python path
sys.path.append(str(basepath.joinpath('src')))

# model implementation
from tcell_model_v7 import *

# plotting settings
from plotting_imports_nb import *

# local settings
exp = 257
datapath = basepath.joinpath(f'expdata/kinetics_exp{exp}')

ct = 'U-D-CTLA4'
celltypes = ['undivided','divided CTLA4-','divided CTLA4+']
cnt0s = [25000,50000,100000]


# Model

Retrieve data from experiments and interpolate that data. The model needs all keys present in `data`.

In [ ]:
def get_interpolated_data(line, well, cnt0):
    data = {}
    ct = 'U-D-CTLA4'
    # FCS data
    fn_expr = datapath.joinpath(f'intensity_for_celltypes_{ct}.csv.gz')
    df_expr = pd.read_csv(fn_expr, sep='\t', compression='gzip')
    sdf_expr = df_expr[(df_expr.line == line) & (df_expr.well == well) & (df_expr.cnt0 == cnt0)]
    data['CD25u'] = sdf_expr[sdf_expr.celltype == 'U'].groupby('time').mean()['CD25']
    data['CD25neg'] = sdf_expr[sdf_expr.celltype == 'Dneg'].groupby('time').mean()['CD25']
    data['CD25pos'] = sdf_expr[sdf_expr.celltype == 'Dpos'].groupby('time').mean()['CD25']
    data['CD80u'] = sdf_expr[sdf_expr.celltype == 'U'].groupby('time').mean()['CD80']
    data['CD80neg'] = sdf_expr[sdf_expr.celltype == 'Dneg'].groupby('time').mean()['CD80']
    data['CD80pos'] = sdf_expr[sdf_expr.celltype == 'Dpos'].groupby('time').mean()['CD80']
    data['CD86u'] = sdf_expr[sdf_expr.celltype == 'U'].groupby('time').mean()['CD86']
    data['CD86neg'] = sdf_expr[sdf_expr.celltype == 'Dneg'].groupby('time').mean()['CD86']
    data['CD86pos'] = sdf_expr[sdf_expr.celltype == 'Dpos'].groupby('time').mean()['CD86']

    # IL-2
    fn_pop = datapath.joinpath(f'processed_data.csv.gz')
    df_pop = pd.read_csv(fn_pop, sep='\t', compression='gzip')
    df_pop.loc[df_pop.well == 'Flat', 'well'] = 'F'
    data['IL2'] = df_pop[(df_pop.line == line) & (df_pop.well == well) &
                         (df_pop.cnt0 == cnt0)].groupby('time').mean()['IL-2']

    for key, df in data.items():
        data[key] = df.dropna()
    return data


Below we set up the rates associated to each arrow in the model. In this way we can easily modify the model rates without modifying the model structure (which is found in `tcell_model_v7.py`. Every rate is function of `state` which is a container, specifically a `namedtuple`, that contains all model states and model inputs. The set up functions take a `Parameter` object as their argument, which is a container class for parameters. 

In [ ]:
def set_up_model_WT_v7(p):
    activ = lambda state: p.p_U
    Uu_death = lambda state: p.d_Uu
    Us_death = lambda state: np.maximum(0, p.d_Us - p.f * state.CD25u * hill(state.IL2, p.n_U, p.k_U))
    p_D_cd25 = lambda state: p.p_D_cd25 * hill(state.IL2, p.n_D_il2, p.k_D_il2)
    w_U = lambda state: np.minimum(1, state.CD80u + state.CD86u) * state.Us
    w_Dneg = lambda state: np.minimum(1, state.CD80neg + state.CD86neg) * state.Dneg
    w_Dpos = lambda state: np.minimum(1, state.CD80pos + state.CD86pos) * state.Dpos
    p_D_cd28 = lambda state: p.p_D_cd28 * hill(w_U(state) + w_Dneg(state) + w_Dpos(state), p.n_D_cd28, p.k_D_cd28) * (
            1 - hill(state.Dpos, p.n_D_ctla4, p.k_D_ctla4))
    Dneg_growth = lambda state: p.p_D + np.minimum(state.CD25neg * p_D_cd25(state) + p_D_cd28(state),
                                                   p.p_D_cd25)
    Dpos_growth = lambda state: p.p_D + np.minimum(state.CD25pos * p_D_cd25(state) + p_D_cd28(state),
                                                   p.p_D_cd25)
    CTLA4_on = lambda state: p.p_Dpos * state.CD25neg * hill(state.IL2, p.n_Dpos, p.k_Dpos)
    CTLA4_off = lambda state: p.p_Dneg
    return ActivationModel(activ, Uu_death, Us_death, Dneg_growth, Dpos_growth, CTLA4_on,
                           CTLA4_off, delay=p.delay)


def set_up_model_DKO_v7(p):
    activ = lambda state: p.p_U
    Uu_death = lambda state: p.d_Uu
    Us_death = lambda state: np.maximum(0, p.d_Us - p.f * state.CD25u * hill(state.IL2, p.n_U, p.k_U))
    p_D_cd25 = lambda state: p.p_D_cd25 * hill(state.IL2, p.n_D_il2, p.k_D_il2)
    Dneg_growth = lambda state: p.p_D + np.minimum(state.CD25neg * p_D_cd25(state),
                                                   state.CD25neg * p.p_D_cd25)
    Dpos_growth = lambda state: p.p_D + np.minimum(state.CD25pos * p_D_cd25(state),
                                                   state.CD25pos * p.p_D_cd25)
    CTLA4_on = lambda state: p.p_Dpos * state.CD25neg * hill(state.IL2, p.n_Dpos, p.k_Dpos)
    CTLA4_off = lambda state: p.p_Dneg
    return ActivationModel(activ, Uu_death, Us_death, Dneg_growth, Dpos_growth, CTLA4_on,
                           CTLA4_off, delay=p.delay)

# Fits

In [ ]:
# scipy solver
solver = 'BDF'

# Run a simulation:
# - set initial state (y0) using values from parameter class (pars)
# - create the model by running the provided set up function (f_setup)
# - run the model using the initial state (y0) and the experimental data
# - collect results as a dataframe and return
def run_sim(t,pars,f_setup,expdata,method='BDF'):
    y0 = [pars.f_unsens*pars.U_0,(1-pars.f_unsens)*pars.U_0,pars.Dneg_0,pars.Dpos_0]
    m = f_setup(pars)
    m.run(y0,t,expdata,method=method)    
    return m.get_df()

In [ ]:
# default parameters, values may be overwritten when included in the fits
pars_def = Parameters(p_U=.1, d_U=.024, p_D=-.13, p_D_cd25=.2, p_D_cd28=.06, p_Dpos=1, p_Dneg=.1,
                      k_D_ctla4=100000, k_D_cd28=9000, k_D_cd25=300, k_D_il2=300, k_Dpos=40,
                      k_Dpos_CD25=4000, k_Dneg=1e-10,
                      n_D_ctla4=2, n_D_cd28=2, n_D_cd25=5, n_D_il2=5, n_Dpos=2, n_Dpos_CD25=3,
                      delay=28, U_0=0, n_U=5, f_unsens=0)

# dataframe with best parameter fits
# note that residuals of the fits are also included
df_pars = pd.read_csv(basepath.joinpath('results','best_fits.csv'),sep='\t')

def plot_fit_results(df_pars,f_setup,line,well,cnt0,cols=None,title=None,method='BDF',t=None,
                     labels=None,sharey=False):     
    exp_col = {'undivided':'cnt undivided','divided CTLA4-':'cnt divided CTLA4-',
           'divided CTLA4+':'cnt divided CTLA4+','cell count':'cnt all'}
    df_ct = pd.read_csv(datapath.joinpath(f'celltypes_{ct}.csv.gz'),
                    sep='\t',compression='gzip')    
    if labels is None:
        labels = len(df_pars)*['_nolegend_']
    if cols is None:
        cols = ['undivided','divided CTLA4-','divided CTLA4+','cell count']        
        ny = 1
        nx = 4
    else:
        ny = int(np.ceil(len(cols)/5))
        nx = int(np.ceil(len(cols)/ny))
    fig,axes = plt.subplots(ny,nx,figsize=(nx*w_im,ny*h_im),sharey=sharey,tight_layout=True) 
    axes = axes.flatten()                
    if title is not None:
        fig.suptitle(title,y=1.05,fontsize=16)
    else:
        fig.suptitle(f'{well} - {cnt0} - {line}',y=1.05,fontsize=16)
    expdata = get_interpolated_data(line, well, cnt0)
    sdf_exp = df_ct[(df_ct.line==line)&(df_ct.well==well)&(df_ct.cnt0==cnt0)]                
    ylim = {}
    for i,col in enumerate(cols):
        if col in exp_col:
            mu = sdf_exp.groupby('time').mean()[exp_col[col]]
            sd = sdf_exp.groupby('time').std()[exp_col[col]]
            axes[i].errorbar(mu.index,mu.values,yerr=sd.values,label='_nolegend_',fmt='o',color='r')
            ylim[col] = axes[i].get_ylim()
    if t is None:
        t = np.linspace(0,84,101)       
    df_pars = df_pars.reset_index()
    for idx,row in df_pars.iterrows():
        pars = dfrow_to_pars(pd.DataFrame(row).transpose(),pars_def)
        df = run_sim(t,pars,f_setup,expdata,method=method) 
        for i,col in enumerate(cols):
            if len(df_pars) <= len(patterns):
                axes[i].plot(df.time,df[col],'-',label=labels[idx],dashes=patterns[idx])                
            else:
                axes[i].plot(df.time,df[col],'-',label=labels[idx])
            axes[i].set(title=col,xlabel='time',xticks=[0,24,48,72,96],xlim=[0,96])
                        #ylim=(axes[i].get_ylim()[0],ylim[col][1]))    
    sns.despine()     
    if len(axes) > len(cols):
        for i in range(len(cols),len(axes)):
            plt.delaxes(axes[i])
    if len(axes[len(cols)-1].get_legend_handles_labels()[0]) > 0:
        axes[len(cols)-1].legend(loc=(1.1,0),frameon=False)

### Flat wells

In [ ]:
well = 'F'
df_best = df_pars[df_pars['well']==well].copy()
for cnt0 in cnt0s:   
    df_best['U_0'] = cnt0
    plot_fit_results(df_best,set_up_model_WT_v7,'WT',well,cnt0,method=solver)
for cnt0 in cnt0s:   
    df_best['U_0'] = cnt0
    plot_fit_results(df_best,set_up_model_DKO_v7,'DKO',well,cnt0,method=solver)

### U wells

In [ ]:
well = 'U'
df_best = df_pars[df_pars['well']==well].copy()
for cnt0 in cnt0s:   
    df_best['U_0'] = cnt0
    plot_fit_results(df_best,set_up_model_WT_v7,'WT',well,cnt0,method=solver)
for cnt0 in cnt0s:   
    df_best['U_0'] = cnt0
    plot_fit_results(df_best,set_up_model_DKO_v7,'DKO',well,cnt0,method=solver)

### V wells

In [ ]:
well = 'V'
df_best = df_pars[df_pars['well']==well].copy()
for cnt0 in cnt0s:   
    df_best['U_0'] = cnt0
    plot_fit_results(df_best,set_up_model_WT_v7,'WT',well,cnt0,method=solver)
for cnt0 in cnt0s:   
    df_best['U_0'] = cnt0
    plot_fit_results(df_best,set_up_model_DKO_v7,'DKO',well,cnt0,method=solver)